In [ ]:
# 2026/1/7
# zhangzhong
# https://docs.langchain.com/oss/python/langgraph/use-subgraphs

In [ ]:
# what is subgraph
# A subgraph is a graph that is used as a node in another graph.

## conmunication between graph and subgraph
# Invoke a graph from a node — subgraphs are called from inside a node in the parent graph
# Add a graph as a node — a subgraph is added directly as a node in the parent and shares state keys with the parent

In [ ]:
## Invoke a graph from a node

# In this case subgraphs can have completely different schemas from the parent graph (no shared keys)

# 这个还是挺常规的，和我们直接调用graph的方式一样，就是 graph.invoke

from typing_extensions import TypedDict
from langgraph.graph.state import StateGraph, START

class SubgraphState(TypedDict):
    bar: str

# Subgraph

def subgraph_node_1(state: SubgraphState):
    return {"bar": "hi! " + state["bar"]}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph = subgraph_builder.compile()

# Parent graph

class State(TypedDict):
    foo: str

def call_subgraph(state: State):
    # Transform the state to the subgraph state
    subgraph_output = subgraph.invoke({"bar": state["foo"]})  
    # Transform response back to the parent state
    return {"foo": subgraph_output["bar"]}

builder = StateGraph(State)
builder.add_node("node_1", call_subgraph)
builder.add_edge(START, "node_1")
graph = builder.compile()

In [ ]:
## Add a graph as a node
# When the parent graph and subgraph can communicate over a shared state key (channel) in the schema, 
# you can add a graph as a node in another graph

from typing_extensions import TypedDict
from langgraph.graph.state import StateGraph, START


# 感觉subgraph的state也可以继承parent的state啊，他们共享foo

# Define subgraph
class SubgraphState(TypedDict):
    foo: str  # shared with parent graph state
    bar: str  # private to SubgraphState

def subgraph_node_1(state: SubgraphState):
    return {"bar": "bar"}

def subgraph_node_2(state: SubgraphState):
    # note that this node is using a state key ('bar') that is only available in the subgraph
    # and is sending update on the shared state key ('foo')
    return {"foo": state["foo"] + state["bar"]}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")

# If you want the subgraph to have its own memory
# This is useful in multi-agent systems, if you want agents to keep track of their internal message histories:
# subgraph = subgraph_builder.compile(checkpointer=True)
subgraph = subgraph_builder.compile()

# Define parent graph
class ParentState(TypedDict):
    foo: str

def node_1(state: ParentState):
    return {"foo": "hi! " + state["foo"]}

builder = StateGraph(ParentState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", subgraph)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
# graph = builder.compile(checkpointer=checkpointer)
graph = builder.compile()

for chunk in graph.stream({"foo": "foo"}):
    print(chunk)

In [ ]:
## nice！！
# You only need to provide the checkpointer when compiling the parent graph. LangGraph will automatically propagate the checkpointer to the child subgraphs.
# 这一点设计的非常好啊

In [ ]:
# streams from subgraph
for chunk in graph.stream(
    {"foo": "foo"},
    subgraphs=True, 
    stream_mode="updates",
):
    print(chunk)

In [ ]:
# view subgraph state
# graph.get_state(config, subgraphs=True).

In [ ]:
# Using with subgraphs called as functions

def node_in_parent_graph(state: State):
    some_code()  # <-- This will re-execute when resumed
    # Invoke a subgraph as a function.
    # The subgraph contains an `interrupt` call.
    subgraph_result = subgraph.invoke(some_input)
    # ...

def node_in_subgraph(state: State):
    some_other_code()  # <-- This will also re-execute when resumed
    result = interrupt("What's your name?")
    # ...

# 那么为了尽可能的防止interrupt的奇怪行为，